In [1]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM

class ScoringLlamaModel(nn.Module):
    def __init__(self, model_id):
        super(ScoringLlamaModel, self).__init__()

        # 기존 LLaMA 모델 로드
        self.llama = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.float32,
            device_map="auto"
        )

        # 기존 LLaMA의 lm_head 제거
        del self.llama.lm_head

        # 새로운 회귀/분류 레이어 추가 (출력: 5개 점수)
        embedding_dim = self.llama.config.hidden_size  # 모델의 임베딩 차원
        self.score_head = nn.Linear(embedding_dim, 5)

    def forward(self, input_ids, attention_mask=None):
        # LLaMA의 마지막 hidden state 가져오기
        outputs = self.llama.model(input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state  # (batch_size, seq_length, embedding_dim)

        # 마지막 토큰의 hidden state만 추출 (CLS 토큰처럼 사용)
        last_token_hidden = hidden_states[:, -1, :]  # (batch_size, embedding_dim)

        # 새로운 score_head를 거쳐 5개 점수 예측
        scores = self.score_head(last_token_hidden)  # (batch_size, 5)
        return scores


In [2]:
model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = ScoringLlamaModel(model_id).to("cuda")

instruction = "Hornworts는 번식을 위해 습기에 의존하는 식물입니다. 이들은 물이 있는 환경에서 생식세포가 서로 결합하여 수정이 이루어지며, 이는 물리적인 매개체가 없으면 불가능합니다. 또한, Hornworts는 환경의 습도에 따라 생장과 번식 주기가 영향을 받기 때문에, 습기는 이들 생물의 생존과 번식에 필수적입니다."

messages = [
    {"role": "system", "content": "논리 문제의 답을 채점하고 창의력, 논리력, 사고력, 설득력, 추론의 깊이의 5개 분야로 나눠서 점수와 근거를 보여줘."},
    {"role": "assistant", "content": "Hornworts는 번식을 위해 무엇에 의존합니까?"},
    {"role": "user", "content": f"{instruction}"},
]

# 입력 데이터 변환
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")

# 모델 예측
scores = model(input_ids)

# 점수 출력
print("Predicted Scores:", scores.tolist())


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

Predicted Scores: [[-0.5033279061317444, 0.5615213513374329, -1.2062736749649048, 0.12753939628601074, -0.1414693295955658]]
